# 2018 Later 5 Years

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2025-01-13  


## 1. Import packages

In [ ]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../../')
sys.path.insert(1,'../../../')
from config import conf as cfg
import rxnrecer as production
from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize(progress_bar=False)
import pandas as pd


FIRST_TIME_RUN = True # For the initial run, please set this flag to True. This will allow the program to download data from UniProt and RHEA, which may take longer depending on your internet speed.

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 192 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. TestSet for Newly 5 Years

In [2]:
ds_test = pd.read_feather(cfg.FILE_DS_TEST)
ds_test.head(3)

,uniprot_id,seq,reaction_id,ec_number,functionCounts,ec_specific_level,isenzyme,label
0,A9JLI2,MLGLQIFTLLSIPTLLYTYEIEPLERTSTPPEKEFGYWCTYANHCR...,-,-,0,0,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,A9JLI3,MRFFSYLGLLLAGLTSLQGFSTDNLLEEELRYWCQYVKNCRFCWTC...,-,-,0,0,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,A9JLI5,MLVIFLGILGLLANQVLGLPTQAEGHLRSTDNPPQEELGYWCTYME...,-,-,0,0,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [22]:
if FIRST_TIME_RUN:
    res_ecrecer = production.step_by_step_prediction(input_data=ds_test[['uniprot_id', 'seq']], Ensemble=True)
    res_ecrecer.to_feather(f'{cfg.FILE_CASE_RESULTS_RXNRECER_REACTION}')
else:
    res_ecrecer = pd.read_feather(f'{cfg.FILE_CASE_RESULTS_RXNRECER_REACTION}')

In [3]:
res_ecrecer = production.step_by_step_prediction(input_data=ds_test.head(30)[['uniprot_id', 'seq']], Ensemble=True)

Step 1: Preparing input data
Step 2: Loading predictive model
Use device: cuda
Step 3: Running prediction on 30 proteins


Predicting reactions: 100%|██████████| 15/15 [00:01<00:00,  8.60it/s]


Transferred model to GPU


100%|██████████| 30/30 [00:01<00:00, 22.67it/s]
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/hpcfs/fhome/shizhenkun/miniconda3/envs/rxnrecer/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Processing Sequences: 100%|██████████| 30/30 [00:00<00:00, 36.

In [4]:
res_ecrecer

,input_id,RXNRECer,RXNRECer_with_prob
0,A9JLI2,-,{'-': 0.999997}
1,A9JLI3,-,{'-': 0.999998}
2,A9JLI5,-,{'-': 1.0}
3,A9JLI7,-,{'-': 0.999997}
4,B5KVH4,-,{'-': 0.999998}
5,C0HLM8,-;RHEA:15801,"{'-': 0.999989, 'RHEA:15801': 0.940841}"
6,C0HLR7,-,{'-': 0.999994}
7,C0HLM9,-,{'-': 0.999941}
8,A9NJG2,-,{'-': 0.999997}
9,C0H4V6,-,{'-': 0.999996}
